# Week1.Homework

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Data

Link to data: [NYC Taxi](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

In [2]:
df_jan = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')
df_feb = pd.read_parquet('data/yellow_tripdata_2022-02.parquet')

In [3]:
df_jan.shape, df_feb.shape

((2463931, 19), (2979431, 19))

In [4]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25


In [6]:
def calculate_duration(df):
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)
    return df

In [7]:
df_jan = calculate_duration(df_jan)
df_feb = calculate_duration(df_feb)

In [8]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [9]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667


In [10]:
print(f'Mean duration: {df_jan.duration.mean()}')
print(f'Std duration: {df_jan.duration.std()}')

Mean duration: 14.212202918831741
Std duration: 46.44530513776802


In [11]:
prev_len = len(df_jan)
prev_len

2463931

In [12]:
df_jan[(df_jan.duration < 1) | (df_jan.duration > 60)].shape

(42491, 20)

In [13]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
curr_len = len(df_jan)
df_jan.shape

(2421440, 20)

In [14]:
print(f'Fraction of obervation left: {curr_len/prev_len:.3f}')

Fraction of obervation left: 0.983


In [15]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [16]:
df_jan.PULocationID.isna().sum(), df_jan.DOLocationID.isna().sum()

(0, 0)

In [17]:
# df_jan.PUlocationID.fillna(-1, inplace=True)
# df_jan.DOlocationID.fillna(-1, inplace=True)

In [18]:
# print(f'PU location NaN fraction: {(df_jan.PUlocationID == -1).sum() * 100 / len(df_jan.PUlocationID):.2f}')
# print(f'DO location NaN fraction: {(df_jan.DOlocationID == -1).sum() * 100 / len(df_jan.DOlocationID):.2f}')

In [19]:
columns = ['PULocationID', 'DOLocationID']

In [20]:
df_jan[columns] = df_jan[columns].astype(str)
jan_dicts = df_jan[columns].to_dict(orient='records')
jan_dicts

[{'PULocationID': '142', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '42'},
 {'PULocationID': '166', 'DOLocationID': '166'},
 {'PULocationID': '114', 'DOLocationID': '68'},
 {'PULocationID': '68', 'DOLocationID': '163'},
 {'PULocationID': '138', 'DOLocationID': '161'},
 {'PULocationID': '233', 'DOLocationID': '87'},
 {'PULocationID': '238', 'DOLocationID': '152'},
 {'PULocationID': '166', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '141'},
 {'PULocationID': '141', 'DOLocationID': '229'},
 {'PULocationID': '114', 'DOLocationID': '90'},
 {'PULocationID': '234', 'DOLocationID': '113'},
 {'PULocationID': '246', 'DOLocationID': '79'},
 {'PULocationID': '43', 'DOLocationID': '140'},
 {'PULocationID': '239', 'DOLocationID': '151'},
 {'PULocationID': '148', 'DOLocationID': '141'},
 {'PULocationID': '237', 'DOLocationID': '107'},
 {'PULocationID': '7', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '263'},
 {'PULocationID': '263', 'DOLoc

In [21]:
dv = DictVectorizer()

X_jan = dv.fit_transform(jan_dicts)
X_jan.shape

(2421440, 515)

In [22]:
X_jan

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [23]:
target = 'duration'
y_jan = df_jan[target].values

In [24]:
lr = LinearRegression()

lr.fit(X_jan, y_jan)
y_pred = lr.predict(X_jan)

In [25]:
mean_squared_error(y_jan, y_pred, squared=False)

6.986191065500608

## Main part

In [26]:
def read_data(filename, categorical):
    df = pd.read_parquet(filename)
    df = calculate_duration(df)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df[categorical] = df[categorical].fillna(-1)
    df[categorical] = df[categorical].astype(str)
    
    return df

In [27]:
def prepare_data(df, dv, columns, is_pretrained=False):
    dicts = df[columns].to_dict(orient='records')
    if not is_pretrained:
        dv.fit(dicts)

    X_ = dv.transform(dicts)
    
    return X_, dv

In [28]:
train_filename = 'data/yellow_tripdata_2022-01.parquet'
val_filename = 'data/yellow_tripdata_2022-02.parquet'

In [29]:
train = read_data(train_filename, columns)
val = read_data(val_filename, columns)

print(train.shape, val.shape)

(2421440, 20) (2918187, 20)


In [30]:
dv = DictVectorizer()
X_train, dv = prepare_data(train, dv, columns)
X_val, _ = prepare_data(val, dv, columns, True)

X_train.shape, X_val.shape

((2421440, 515), (2918187, 515))

In [31]:
target = 'duration'
y_train = train[target].values
y_val = val[target].values

In [32]:
lr = LinearRegression()

lr.fit(X_train, y_train)
y_pred_tr = lr.predict(X_train)
y_pred_val = lr.predict(X_val)

In [33]:
print(f'Train RMSE: {mean_squared_error(y_train, y_pred_tr, squared=False)}')

Train RMSE: 6.986191065500608


In [34]:
print(f'Val RMSE: {mean_squared_error(y_val, y_pred_val, squared=False)}')

Val RMSE: 7.786408015215065
